# Refactor bus corridors

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(900_000_000_000) ## 800GB?

import datetime as dt
import geopandas as gpd
import pandas as pd
import zlib

from siuba import *

import utilities
import A1_rail_ferry_brt as rail_ferry_brt

from shared_utils import rt_utils

GCS_FILE_PATH = utilities.GCS_FILE_PATH

analysis_date = rail_ferry_brt.analysis_date
analysis_date

import dask.dataframe as dd
import numpy as np

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
# calitp-analytics-data/data-analyses/high_quality_transit_areas/bus_corridors/182_bus.parquet
bus_corridor = gpd.read_parquet(f"{GCS_FILE_PATH}bus_corridors/182_bus.parquet")

In [ ]:
#bus_corridor2 = gpd.read_parquet("./data/182_bus2.parquet")

In [2]:
itp_id = 182
date_str = analysis_date.strftime(rt_utils.FULL_DATE_FMT)

# Skip writing geoparquet again for now 
# TODO: tweak rt_utils to overwrite export? 
# Overwriting while testing this is not ideal, don't want to mess it up

#routelines = rt_utils.get_routelines(itp_id, analysis_date)
#routelines = gpd.read_parquet(f"{rt_utils.GCS_FILE_PATH}"
#                             f"cached_views/routelines_{itp_id}_{date_str}.parquet" 
#                            )

#routelines.to_parquet(f"./data/routelines_{itp_id}_{date_str}.parquet")

## force clear to ensure route type data present
#trips = rt_utils.get_trips(itp_id, analysis_date, force_clear=True, route_types = ['3'])
#trips.to_parquet(f"./data/trips_{itp_id}_{date_str}.parquet")

#stop_times = rt_utils.get_stop_times(itp_id, analysis_date)
#stop_times.to_parquet(f"./data/st_{itp_id}_{date_str}.parquet")

#stops = rt_utils.get_stops(itp_id, analysis_date)
#stops.to_parquet(f"./data/stops_{itp_id}_{date_str}.parquet")

In [3]:
routelines = gpd.read_parquet(f"./data/routelines_{itp_id}_{date_str}.parquet")
#trips = pd.read_parquet(f"./data/trips_{itp_id}_{date_str}.parquet")
#stop_times = pd.read_parquet(f"./data/st_{itp_id}_{date_str}.parquet")
#stops = gpd.read_parquet(f"./data/stops_{itp_id}_{date_str}.parquet")

In [ ]:
#import dask_utils

#peak_trips_by_stop = dask_utils.stop_times_aggregation(itp_id, analysis_date)

In [4]:
import dask_geopandas

In [5]:
gddf = dask_geopandas.from_geopandas(routelines, npartitions=4)

In [12]:
gddf = gddf.assign(
    **{"shape_id_length": gddf.geometry.length,
        "geometry": gddf.geometry.buffer(50),
      }
).sort_values("shape_id_length", ascending=False)

In [20]:
gddf.head(100).shape_id_length.sum()

4068548.9483745582

In [21]:
gddf.shape_id_length.sum().compute()

16790974.473367132

In [22]:
4068548/16790974

0.24230565778971488

In [ ]:
dissolved = routelines.dissolve()[["calitp_itp_id", "calitp_url_number", "geometry"]]


In [ ]:
dissolved.crs

In [ ]:
len(routelines)

In [ ]:
#dissolved_segmented = utilities.create_segments(dissolved["geometry"])

# Turn 1 row of geometry into hqta_segments, every 1,250 m
segmented = gpd.GeoDataFrame() ##changed to gdf?

for segment in utilities.create_segments(dissolved.geometry):
    to_append = dissolved.drop(columns=["geometry"])
    to_append["geometry"] = segment
    segmented = pd.concat((segmented, to_append))

In [ ]:
segmented["segment_sequence"] = segmented.reset_index(drop=True).index

In [ ]:
dissolved.plot()

In [ ]:
gddf2 = gddf.head(10)
gddf2.shape_id_length.sum().compute()

## All shapes for operator
* Dissolve by length, and generate segments at once
* Join stops once